# UT10: Proyecto Final Integrador

Análisis de datos con Python

# UT10: Proyecto Final Integrador - De Datos a Decisiones

> **Cuaderno de trabajo — UT10: Proyecto Final Integrador**
>
> Este notebook es tu lienzo en blanco. Contiene las instrucciones del
> proyecto. Para el contexto completo, advertencias y consejos
> profesionales, consulta el libro (PDF).

## Checklist para Portfolio y Entrevista Técnica

En lugar de una rúbrica escolar, utiliza este *checklist* profesional.
Si superas todas estas preguntas, tienes un proyecto que puedes subir a
tu perfil de GitHub y defender con orgullo en una entrevista para tu
primer empleo como Data Analyst o Junior Data Scientist.

**Checklist de validación final**

Antes de dar el proyecto por terminado, revisa tu código y asegúrate de
poder responder “Sí” a todas estas cuestiones operativas y de negocio:

| Dimensión               | Pregunta de validación                                                                                                                                             |
|:-----------------------------------|:-----------------------------------|
| **Código y Estilo**     | ¿He evitado los bucles `for` siempre que ha sido posible usando operaciones vectorizadas o `.apply()`?                                                             |
| **Código y Estilo**     | ¿Está mi código organizado en funciones claras y he usado el *method chaining* (`.pipe`, `.assign`) de Pandas?                                                     |
| **Ingeniería de Datos** | ¿Encontré el error de tipo en `TotalCharges`, documenté mi decisión de cómo tratarlo, y no me quedaron nulos en el *dataset* final?                                |
| **EDA de Negocio**      | ¿Mis gráficos tienen títulos descriptivos (ej: *“La fibra óptica duplica el riesgo de abandono”*) en lugar de títulos técnicos (ej: *“InternetService vs Churn”*)? |
| **Rigor en ML**         | ¿Puedo demostrar que hice el *Split Train/Test* **antes** de escalar las variables, evitando así el *Data Leakage*?                                                |
| **Rigor en ML**         | ¿Utilicé `stratify=y` al dividir los datos para lidiar con el desbalanceo del *dataset*?                                                                           |
| **Interpretación**      | Si un entrevistador me pregunta *“¿Por qué no has usado el Accuracy para evaluar el modelo?”*, ¿sé explicarle basándome en los Falsos Negativos y el *Recall*?     |
| **Producto (App)**      | ¿La aplicación carga rápido (usando `@st.cache_data`) y no se “rompe” al cambiar los selectores en la barra lateral?                                               |

**Tiempo estimado de realización del proyecto:** Entre 15 y 20 horas de
trabajo autónomo.

``` python
# Este es tu espacio. ¡Crea algo de lo que te sientas orgulloso!
```